In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import mord as m
import seaborn as sns
import numpy as np
import torch.nn as nn
from torch.nn import Conv2d, MaxPool2d, Parameter
from torch.nn.functional import relu
import torch

In [30]:
df = pd.read_csv("clean_data/movies.csv")

columns = ["popularity", "revenue", "budget", "runtime", "vote_average"]
df = df.fillna(0)
df = df[columns]
df["label"] = df["vote_average"].apply(lambda x: 1 if x >= 5 else 0)
df.drop(["vote_average"], axis=1, inplace=True)

def prep_data(df):
    y = torch.tensor(df["label"].values)
    X = df.drop(["label"], axis = 1)
    X = torch.tensor(X.values)

    return X, y

X, y = prep_data(df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
class LinearModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()

        self.pipeline = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, 1),
            nn.ReLU(),
        )

    # this is the customary name for the method that computes the scores
    # the loss is usually computed outside the model class during the training loop
    def forward(self, x):
        return self.pipeline(x)

# Get the number of features from X_train
input_dim = X_train.shape[1]

# Initialize the model with the correct input dimension
model = LinearModel(input_dim)

In [41]:
scores = model(X_train.float())  # Ensure X_train is of type float
print(scores)

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)


I did some very basic setup for what will become a deep learning model to analyze the movie data. I started with a simple linear model so that I have some of the structure in place for the future. I also improved my data prep process to involve torch tensors.